<a href="https://colab.research.google.com/github/ankersp/thesis/blob/main/BERT%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers accelerate datasets --quiet
!pip install fsspec==2024.10.0 --quiet

# Import required libraries
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Hugging Face Transformers and Datasets
import datasets
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    pipeline,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have f

In [2]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# ESG classification

In [7]:
import pandas as pd

# Specify the path to the file in Google Drive
file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/sentences_long_format.csv'
data = pd.read_csv(file_path)


In [8]:
from transformers import pipeline

# Specify the paths to the fine-tuned models
env_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_env_model"
soc_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_soc_model"
gov_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_gov_model"

# Load pipelines for each ESG model
env_pipeline = pipeline(
    'text-classification',
    model=env_model_path,
    tokenizer=env_model_path,
    device=0
)

soc_pipeline = pipeline(
    'text-classification',
    model=soc_model_path,
    tokenizer=soc_model_path,
    device=0
)

gov_pipeline = pipeline(
    'text-classification',
    model=gov_model_path,
    tokenizer=gov_model_path,
    device=0
)

# Verify the pipelines are loaded
print("EnvironmentalBERT pipeline loaded.")
print("SocialBERT pipeline loaded.")
print("GovernanceBERT pipeline loaded.")


Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


EnvironmentalBERT pipeline loaded.
SocialBERT pipeline loaded.
GovernanceBERT pipeline loaded.


In [9]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Paths
data_file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/sentences_long_format.csv'
save_file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/esg_classifications.csv'

# Model paths
env_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_env_model"
soc_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_soc_model"
gov_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_gov_model"

# Load dataset
data = pd.read_csv(data_file_path)

# Load existing results or start fresh
try:
    df_saved_results = pd.read_csv(save_file_path)
    print(f"Loaded saved results. Documents already processed: {df_saved_results['id'].nunique()}")
except FileNotFoundError:
    print("No previous results found. Starting fresh.")
    df_saved_results = pd.DataFrame(columns=['id', 'text', 'env_label', 'env_score', 'soc_label', 'soc_score', 'gov_label', 'gov_score'])

# Identify sentences that still need to be classified
processed_ids = df_saved_results['id'].unique()
remaining_data = data[~data['id'].isin(processed_ids)]

# Load ESG pipelines
env_pipeline = pipeline("text-classification", model=env_model_path, tokenizer=env_model_path, device=0)
soc_pipeline = pipeline("text-classification", model=soc_model_path, tokenizer=soc_model_path, device=0)
gov_pipeline = pipeline("text-classification", model=gov_model_path, tokenizer=gov_model_path, device=0)

# Function to classify ESG labels and save results
def classify_esg(data, df_saved_results, env_pipeline, soc_pipeline, gov_pipeline, save_interval=1000, save_file_path=save_file_path):
    all_results = []

    try:
        for start_idx in tqdm(range(0, data.shape[0], 32), desc="Classifying ESG"):
            batch = data.iloc[start_idx:start_idx + 32]
            sentences = batch['sentences'].tolist()
            ids = batch['id'].tolist()

            # Classify batch for Environmental
            env_classifications = env_pipeline(sentences, padding=True, truncation=True)

            # Classify batch for Social
            soc_classifications = soc_pipeline(sentences, padding=True, truncation=True)

            # Classify batch for Governance
            gov_classifications = gov_pipeline(sentences, padding=True, truncation=True)

            for i in range(len(sentences)):
                row_data = {
                    "id": ids[i],
                    "text": sentences[i],
                    "env_label": env_classifications[i]["label"],
                    "env_score": env_classifications[i]["score"],
                    "soc_label": soc_classifications[i]["label"],
                    "soc_score": soc_classifications[i]["score"],
                    "gov_label": gov_classifications[i]["label"],
                    "gov_score": gov_classifications[i]["score"]
                }
                all_results.append(row_data)

            # Save intermediate results
            if (start_idx // 32 + 1) % (save_interval // 32) == 0:
                print(f"Saving progress at batch {start_idx // 32 + 1}...")
                df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
                df_saved_results.to_csv(save_file_path, index=False)
                all_results = []

        # Final save
        if all_results:
            df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
            df_saved_results.to_csv(save_file_path, index=False)
            print(f"Final results saved to {save_file_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return df_saved_results

# Run classification
if remaining_data.shape[0] > 0:
    df_saved_results = classify_esg(remaining_data, df_saved_results, env_pipeline, soc_pipeline, gov_pipeline)
else:
    print("All sentences have already been classified.")


No previous results found. Starting fresh.


Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Classifying ESG:   0%|          | 30/6169 [00:15<50:45,  2.02it/s]<ipython-input-9-4b9b46566c96>:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
Classifying ESG:   1%|          | 31/6169 [00:16<50:30,  2.03it/s]

Saving progress at batch 31...


Classifying ESG:   1%|          | 62/6169 [00:32<51:04,  1.99it/s]

Saving progress at batch 62...


...

Classifying ESG:  99%|█████████▉| 6137/6169 [57:28<00:15,  2.08it/s]

Saving progress at batch 6138...


Classifying ESG: 100%|█████████▉| 6168/6169 [57:46<00:00,  1.77it/s]

Saving progress at batch 6169...


Classifying ESG: 100%|██████████| 6169/6169 [57:49<00:00,  1.78it/s]


# Action and specificity classifications

In [3]:
import pandas as pd

# Specify the path to the file in Google Drive
file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/esg_classifications.csv'
data = pd.read_csv(file_path)


In [4]:
from transformers import pipeline

# Specify the paths to the fine-tuned models
act_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_action_model"
specific_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_climatebert_specificity_model"

# Load pipelines for each ESG model
act_pipeline = pipeline(
    'text-classification',
    model=act_model_path,
    tokenizer=act_model_path,
    device=0
)

specific_pipeline = pipeline(
    'text-classification',
    model=specific_model_path,
    tokenizer=specific_model_path,
    device=0
)

# Verify the pipelines are loaded
print("action pipeline loaded.")
print("specificity pipeline loaded.")


Device set to use cuda:0
Device set to use cuda:0


action pipeline loaded.
specificity pipeline loaded.


In [5]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# Paths
data_file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/esg_classifications.csv'
save_file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/environmental_action_specificity.csv'

# Model paths
act_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_esgbert_action_model"
specific_model_path = "/content/drive/My Drive/KU/kandidat/speciale/R/finetuned_climatebert_specificity_model"

# Load dataset
data = pd.read_csv(data_file_path)

# Filter only environmental sentences
env_data = data[data['env_label'] == 'LABEL_1']

# Load existing results or start fresh
try:
    df_saved_results = pd.read_csv(save_file_path)
    print(f"Loaded saved results. Sentences already processed: {df_saved_results['id'].nunique()}")
except FileNotFoundError:
    print("No previous results found. Starting fresh.")
    df_saved_results = pd.DataFrame(columns=['id', 'text', 'action_label', 'action_score', 'specificity_label', 'specificity_score'])

# Identify sentences that still need to be classified
processed_ids = df_saved_results['id'].unique()
remaining_data = env_data[~env_data['id'].isin(processed_ids)]

# Load pipelines
act_pipeline = pipeline('text-classification', model=act_model_path, tokenizer=act_model_path, device=0)
specific_pipeline = pipeline('text-classification', model=specific_model_path, tokenizer=specific_model_path, device=0)

# Function to classify sentences with action and specificity models
def classify_sentences(data, df_saved_results, act_pipeline, specific_pipeline, save_interval=1000, save_file_path=save_file_path):
    all_results = []

    try:
        for start_idx in tqdm(range(0, data.shape[0], 32), desc="Classifying Action and Specificity"):
            batch = data.iloc[start_idx:start_idx + 32]
            sentences = batch['text'].tolist()
            ids = batch['id'].tolist()

            # Classify batch for Action
            action_classifications = act_pipeline(sentences, padding=True, truncation=True)

            # Classify batch for Specificity
            specificity_classifications = specific_pipeline(sentences, padding=True, truncation=True)

            for i in range(len(sentences)):
                row_data = {
                    "id": ids[i],
                    "text": sentences[i],
                    "action_label": action_classifications[i]["label"],
                    "action_score": action_classifications[i]["score"],
                    "specificity_label": specificity_classifications[i]["label"],
                    "specificity_score": specificity_classifications[i]["score"]
                }
                all_results.append(row_data)

            # Save intermediate results
            if (start_idx // 32 + 1) % (save_interval // 32) == 0:
                print(f"Saving progress at batch {start_idx // 32 + 1}...")
                df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
                df_saved_results.to_csv(save_file_path, index=False)
                all_results = []

        # Final save
        if all_results:
            df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
            df_saved_results.to_csv(save_file_path, index=False)
            print(f"Final results saved to {save_file_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

    return df_saved_results

# Run classification
if remaining_data.shape[0] > 0:
    df_saved_results = classify_sentences(remaining_data, df_saved_results, act_pipeline, specific_pipeline)
else:
    print("All sentences have already been classified.")


No previous results found. Starting fresh.


Device set to use cuda:0
Device set to use cuda:0
Classifying Action and Specificity:   2%|▏         | 30/1776 [00:12<09:52,  2.95it/s]<ipython-input-5-7fd0d6392cae>:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_saved_results = pd.concat([df_saved_results, pd.DataFrame(all_results)], ignore_index=True)
Classifying Action and Specificity:   2%|▏         | 31/1776 [00:12<10:09,  2.86it/s]

Saving progress at batch 31...


Classifying Action and Specificity:   3%|▎         | 62/1776 [00:24<10:27,  2.73it/s]

Saving progress at batch 62...


...

Saving progress at batch 1736...


Classifying Action and Specificity:  99%|█████████▉| 1766/1776 [11:26<00:04,  2.10it/s]

Saving progress at batch 1767...


Classifying Action and Specificity: 100%|██████████| 1776/1776 [11:30<00:00,  2.57it/s]


Final results saved to /content/drive/My Drive/KU/kandidat/speciale/R/environmental_action_specificity.csv
